# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
experiment = 'DDB_PD_117_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,growth_rate,Feed Medium #1,Reactor/Plate/Flask Number,...,Culture Volume Unit,Temperature Setpoint (C),Feed #1 Profile,pH Setpoint,pH Control Base Solution,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type
0,DDB_PD_117_AMBR_DDB35-005_R05_Main_,DDB_BM_016,R05,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R05,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air
1,DDB_PD_117_AMBR_DDB35-005_R06_Main_,DDB_BM_016,R06,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R06,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air
2,DDB_PD_117_AMBR_DDB35-005_R07_Main_,DDB_BM_016,R07,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R07,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air
3,DDB_PD_117_AMBR_HMP3071-003_R11_Main_,DDB_BM_016,R11,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R11,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air
4,DDB_PD_117_AMBR_HMP3071-003_R12_Main_,DDB_BM_016,R12,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R12,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air
5,DDB_PD_117_AMBR_HMP3071-003_R13_Main_,DDB_BM_016,R13,AMBR 250,No,Main,DDB_PD_117_AMBR,0.05,DDB_FM_015,R13,...,mL,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,Temperature Setpoint (C),Feed #1 Profile,pH Setpoint,pH Control Base Solution,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Phase
0,0.000000,99.384914,6.529322,1002.000000,29.989540,0.547836,100.050691,NaN,20.95,20.702183,...,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air,batch
1,0.033333,96.616297,6.541942,999.000000,29.940001,2.818765,100.142318,NaN,20.95,20.699981,...,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air,batch
2,0.066667,96.369291,6.537188,1000.924727,29.910376,2.575785,99.961470,NaN,20.95,20.699963,...,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air,batch
3,0.100000,96.380072,6.541825,999.824940,29.900875,2.846688,100.052309,NaN,20.95,20.698999,...,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air,batch
4,0.133333,96.283285,6.543865,1002.595576,29.920000,2.440766,99.970912,NaN,20.95,20.698999,...,30,DDB_Feed_Profile_DDB_PD_117_AMBR,6.5,Ammonium hydroxide 8%,0.15,0.15,4000,4000,Air,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_117_AMBR_R05__S01,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S01,12.085309,7.0,13.49
1,DDB_PD_117_AMBR_R05__S02,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S02,15.372317,5.0,36.85
2,DDB_PD_117_AMBR_R05__S03,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S03,35.501496,3.5,88.00
3,DDB_PD_117_AMBR_R05__S04,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S04,55.554583,3.5,164.60
4,DDB_PD_117_AMBR_R05__S05,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S05,75.579442,7.5,280.80


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
27,DDB_PD_117_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,2023-01-17,12.085309,DDB_PD_117_AMBR_R05__S01,citric acid,0.433204,g/L
28,DDB_PD_117_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,2023-01-17,12.085309,DDB_PD_117_AMBR_R05__S01,acetic acid,0.771709,g/L
29,DDB_PD_117_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,2023-01-17,12.085309,DDB_PD_117_AMBR_R05__S01,formic acid,0.582330,g/L
30,DDB_PD_117_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,2023-01-17,12.085309,DDB_PD_117_AMBR_R05__S01,pyruvic acid,0.000000,g/L
31,DDB_PD_117_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,2023-01-17,12.085309,DDB_PD_117_AMBR_R05__S01,potassium dihydrogen phosphate,11.311882,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],tryptophan [g/L],Timepoint Sample
0,11.143793,0.771709,0.433204,0.139903,0.58233,0.330215,11.311882,0.0,0.000000,NaN,DDB_PD_117_AMBR_R05__S01
1,0.000000,0.292768,0.445241,0.000000,0.00000,0.000000,10.999189,0.0,0.000000,NaN,DDB_PD_117_AMBR_R05__S02
2,0.000000,0.000000,0.288230,0.000000,0.00000,0.000000,8.292849,0.0,0.000000,NaN,DDB_PD_117_AMBR_R05__S03
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,5.798261,0.0,0.000000,NaN,DDB_PD_117_AMBR_R05__S04
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.104975,2.972819,0.0,0.348699,NaN,DDB_PD_117_AMBR_R05__S05


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,Timepoint Sample


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample,CDW_1 [g/L],CDW_2 [g/L],CDW_3 [g/L]
0,DDB_PD_117_AMBR_R05__S01,4.50,4.50,4.30
1,DDB_PD_117_AMBR_R05__S02,10.60,10.70,10.00
2,DDB_PD_117_AMBR_R05__S03,25.25,25.00,24.75
3,DDB_PD_117_AMBR_R05__S04,52.60,52.40,52.80
4,DDB_PD_117_AMBR_R05__S05,92.30,92.30,91.80
5,DDB_PD_117_AMBR_R06__S01,4.50,4.90,4.40
6,DDB_PD_117_AMBR_R06__S02,10.00,10.70,9.90
7,DDB_PD_117_AMBR_R06__S03,25.75,24.75,26.50
8,DDB_PD_117_AMBR_R06__S04,53.60,53.80,54.40
9,DDB_PD_117_AMBR_R06__S05,93.20,92.80,92.60


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],...,ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],tryptophan [g/L],CDW_1 [g/L],CDW_2 [g/L],CDW_3 [g/L]
0,DDB_PD_117_AMBR_R05__S01,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S01,12.085309,7.0,13.49,11.143793,0.771709,...,0.139903,0.58233,0.330215,11.311882,0.0,0.000000,NaN,4.50,4.5,4.30
1,DDB_PD_117_AMBR_R05__S02,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S02,15.372317,5.0,36.85,0.000000,0.292768,...,0.000000,0.00000,0.000000,10.999189,0.0,0.000000,NaN,10.60,10.7,10.00
2,DDB_PD_117_AMBR_R05__S03,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S03,35.501496,3.5,88.00,0.000000,0.000000,...,0.000000,0.00000,0.000000,8.292849,0.0,0.000000,NaN,25.25,25.0,24.75
3,DDB_PD_117_AMBR_R05__S04,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S04,55.554583,3.5,164.60,0.000000,0.000000,...,0.000000,0.00000,0.000000,5.798261,0.0,0.000000,NaN,52.60,52.4,52.80
4,DDB_PD_117_AMBR_R05__S05,DDB_PD_117_AMBR,DDB_PD_117_AMBR_DDB35-005_R05_Main_,R05,S05,75.579442,7.5,280.80,0.000000,0.000000,...,0.000000,0.00000,0.104975,2.972819,0.0,0.348699,NaN,92.30,92.3,91.80


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)